In [171]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import http.client

In [172]:
import requests

In [173]:
df = pd.read_excel("behaviour_simulation_train.xlsx")

In [174]:
df.head()

,id,date,likes,content,username,media,inferred company
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent


In [175]:
df.isnull().sum()

id                  0
date                0
likes               0
content             0
username            0
media               0
inferred company    0
dtype: int64

In [176]:
print("Username=TimHortonsPH? : ",df[df["username"]=="TimHortonsPH"].count()['id'])

Username=TimHortonsPH? :  185


In [177]:
print("Username=TimHortons? : ",df[df["username"]=="TimHortonsPH"].count()['id'])

Username=TimHortons? :  185


In [178]:
print("How many in total? : ", df['id'].count())

How many in total? :  300000


In [179]:
print("Inferred brand=TimHortons? : ", df[df["inferred company"]=="tim hortons"].count()['id'])

Inferred brand=TimHortons? :  807


In [180]:
print("Inferred brand=Independent? : ", df[df["inferred company"]=="independent"].count()['id'])

Inferred brand=Independent? :  39248


In [181]:
print("How many unique inferred companies? : ", len(df['inferred company'].unique()))

How many unique inferred companies? :  220


In [182]:
print("Unique usernames? : ", len(df['username'].unique()))

Unique usernames? :  2449


In [183]:
# Replace inferred company name and see all usernames
comp = df[df["inferred company"]=="cnn"]
"Number: "+str(len(comp["username"].unique())),comp["username"].unique()

TypeError: 'str' object is not callable

# Videos

In [184]:
dfrev = df.iloc[::-1]

In [185]:
videos_df = dfrev[dfrev['media'].str.startswith("[Video")]

In [186]:
videos_df.count()

id                  77146
date                77146
likes               77146
content             77146
username            77146
media               77146
inferred company    77146
dtype: int64

In [170]:
vidids=[id for id in videos_df['id'].values]
result_string = ', '.join(['{}'.format(num) for num in vidids[::-1]])
with open("vidIDs.txt",'w') as vidIDs:
    vidIDs.write('['+result_string+']')

In [189]:
with open('log_video2.txt', 'w'):
    pass

In [190]:
indices=[]

for i in range(51600,len(videos_df)): # (26000,51600) (51600,len(videos_df))
    string=videos_df.iloc[i]['media']
    arr =string.split(",")
    for s in arr:
        if(s.find('http')!=-1):
            ss=s[s.find("http"):-1]
            if(ss.find("video.")!=-1 and ss.find(".m3u8")==-1):
                with open('log_video2.txt', 'a') as log_file:
                    # print(ss)
                    try:
                        url_parts = http.client.urlsplit(ss)
                        host = url_parts.netloc
                        path = url_parts.path or '/'
                        connection = http.client.HTTPConnection(host)
                        connection.request("GET", path)
                        response = connection.getresponse()
                        if(response.status<400):
                            log_file.write(f"Success! Status Code: {response.status}"+'\n')
                            break
                        else:
                            log_file.write(f"Error! Response Code: {response.status}"+'\n')
                            indices.append(i)
                            break

                    except http.client.HTTPException as e:
                        indices.append(i)
                        log_file.write(f"Error: {e}" + '\n')
                        break